In [1]:
import Transform.Schedule

from Configuration import Editor

with Editor('Config') as Config:

    # settings for dataset
    Config.Dataset.ImagesRootPath = r'D:\Dataset_Collection\Cardiac_Catheterization\train\images'
    Config.Dataset.MasksRootPath = r'D:\Dataset_Collection\Cardiac_Catheterization\train\masks'
    # control input and output image format
    Config.Dataset.IO.InputRGBImage = False
    Config.Dataset.IO.NumWorkers = 0
    Config.Dataset.IO.PinMemory = False
    Config.Dataset.IO.PrefetchFactor = 2
    Config.Dataset.IO.OutputDtype = 'float'
    # uniformed preprocess
    Config.Dataset.Preprocess.Version = 'v1'
    #train dataset
    Config.Dataset.Train.BatchSize = 1
    Config.Dataset.Train.Transform.Combination.Version = 'v1'
    Config.Dataset.Train.Transform.Schedule = 0.8
    Config.Dataset.Train.Transform.Combination.Components = 'default'
    Config.Dataset.Train.Transform.Combination.Params = 'default'
    Config.Dataset.Train.Transform.Combination.Schedules = 'default'
    #validation dataset
    Config.Dataset.Validation.BatchSize = 2
    Config.Dataset.Validation.Ratio = 0.05
    Config.Dataset.Validation.Transform.Combination.Version = 'v1'
    Config.Dataset.Validation.Transform.Schedule = 0.8
    Config.Dataset.Validation.Transform.Combination.Components = 'default'
    Config.Dataset.Validation.Transform.Combination.Params = 'default'
    Config.Dataset.Validation.Transform.Combination.Schedules = 'default'
    
    #choose training structure, including the model, loss, metrics, optimizer, schedular
    Config.Training.Structure.Type = 'SimpleSeg'
    # model
    Config.Training.Structure.Model.Backbone.Name = 'Backbone1'
    Config.Training.Structure.Model.Backbone.Param = dict(ues_instance_norm=True)
    Config.Training.Structure.Model.Head.Name = 'V1'
    Config.Training.Structure.Model.Head.Param = dict(logit_output=True,in_channels=4)
    # loss
    Config.Training.Structure.Loss.Name = 'DiceBCELoss'
    Config.Training.Structure.Loss.Param = dict(use_logit=True,w_bce=0.2)
    # optimizer
    Config.Training.Structure.Optimizer.Name = 'Adam'
    Config.Training.Structure.Optimizer.Param = dict(lr=0.001)
    # scheduler
    Config.Training.Structure.Scheduler.Name = 'CustomSchedule1'
    Config.Training.Structure.Scheduler.Param = dict(warmup_epochs=1,reduce_gamma=-2)
    # metrics
    Config.Training.Structure.Metrics.Name = ['DiceBCELoss']
    Config.Training.Structure.Metrics.Param = [dict(use_logit=True,w_bce=0.2)]

    # setup training
    Config.Training.Checkpoint.Path = None
    Config.Training.Checkpoint.FileName = ''
    Config.Training.Checkpoint.Resume.Process = False
    Config.Training.Checkpoint.Resume.Optimizer = False
    Config.Training.Checkpoint.Resume.Scheduler = False
    
    # whether to freeze backbone
    Config.Training.Settings.Model.FreezeBackbone = True
    
    # setup training process
    Config.Training.Settings.Epochs = 20
    Config.Training.Settings.GradientAccumulation = 4
    Config.Training.Settings.AmpScaleTrain = True
    # set random property
    Config.Training.Settings.Random.cuDNN.Deterministic = True
    Config.Training.Settings.Random.cuDNN.Benchmark = True
    Config.Training.Settings.Random.Seed.Dataset.Split = 4
    Config.Training.Settings.Random.Seed.Dataset.Transform = 10
    Config.Training.Settings.Random.Seed.Dataset.Shuffle = 6
    Config.Training.Settings.Random.Seed.Model = 99
    
    Config.Logging.StepsPerLog = 8
    Config.Logging.Image.Columns = 3
    Config.Logging.Image.Rows = 10
    Config.Logging.Image.Figsize = (300,300)
    Config.Logging.Image.Fontsize = 200
    Config.Logging.Image.DPI = 10
    Config.Logging.Image.MaskAlpha = 0.6
    
    Config.Logging.RootPath = 'logging'
    Config.Logging.Model.Reference = 'Resnet'
    Config.Logging.Model.Derivative = 'WithRegionalAttention'
    Config.Logging.Model.Branch = 'instance_norm02'
    Config.Logging.Comment = 'DiceBCELoss'
    Config.Logging.Purpose = 'None'
    Config.Logging.Note = 'None'

In [2]:
%run main.py

EPOCH: 8 Training:  80%|███████▉  | 1298/1631 [04:08<01:06,  5.03it/s]